In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, RobustScaler
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
# from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from keras import Sequential
# from keras.layers import Dense, Dropout

## EDA

In [ ]:
pd.options.display.max_columns=200
pd.options.display.max_rows=200

In [ ]:
train=pd.read_csv('/kaggle/input/mentality/train.csv')
test=pd.read_csv('/kaggle/input/mentality/test.csv')
sub=pd.read_csv('/kaggle/input/mentality/sample_submission.csv')

In [ ]:
display(train,test,sub)

In [ ]:
train['voted'].value_counts()

In [ ]:
print(train[train['familysize']>99].index,train[train['familysize']>99]['familysize'].values)

In [ ]:
train[train['familysize']>99]

In [ ]:
# train.loc[379,'familysize'] = 100
train.loc[24598,'familysize'] = 500
# train.drop([379,24598], inplace=True)

In [ ]:
data=train[[i for i in train.columns[train.dtypes!='object'] if 'E' not in i][1:]]
# data['Class']=train['MRC_ID_DI']

plt.style.use('ggplot')
f, ax = plt.subplots(figsize = (15, 30))

ax.set_facecolor('#fafafa')
# ax.set(xlim = (-5, 5)) # X축 범위
plt.ylabel('Variables')
plt.title("Overview Data Set")
ax = sns.boxplot(data = data, orient = 'h',  palette = 'Set2')

## preprocessing

In [ ]:
alldata=pd.concat([train,test],ignore_index=True, sort=False)
alldata

In [ ]:
alldata2=alldata.drop(['index','voted'],1)
le=LabelEncoder()
for i in alldata.columns[alldata.dtypes=='object'] :
    alldata2[i]=le.fit_transform(alldata2[i])

#### 결측치 > family size

# 새로 만든 변수에 age, education 등 합성해보기

## feature engineering

- 질문은 0~39 컬러



1. 응답자 성향 파악
  - m_E : 응답자의 평균 응답시간
  - m_A : 응답자의 평균 선호도
  - dfm(different from mean) : (각 질문별 응답자의 선호도 분포 비율) * (해당 분포의 응답시간 평균 - 응답자의 응답시간)
  - personality : tp(1~10) (부정의 역 + 긍정).sum()
  - late_E, early_E : 응답시간 상위 하위 10% 갯수
2. 언어능력
  - voca : wr합 - wf합
3. 설문조사 성실도(이 이상치는 실제 테스트에서도 나올 수 있기 때문에 사용하지 않음)
  - engnat(모국어)가 영어(1)인 사람 중 wf(허구의 단어)가 있다고 응답한 사람 주의, yes와 no 각각 응답자의 설문 시간 및 3번 선호도의 갯수로 확인
  - wf의 yes 갯수만큼 가중치줘서 의미없게 낮추기
4. age 데이터 활용
  - 나이대 별 응답시간, 선호도 평균

#### 1. 응답자 성향 파악

In [ ]:
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]   # - 값
for flip in flipping_columns: 
    alldata2[flip] = 6 - alldata2[flip]
    
flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]  # 숨겨진 질문 중 - 값
for flip in flipping_secret_columns: 
    alldata2[flip] = 6 - alldata2[flip]

* m_E

In [ ]:
E=[]
for i in alldata2.columns:
    if 'E' in i:
        E.append(i)
m_E=alldata2[E].mean(axis=1)

# alldata2['m_E']=m_E
# alldata2

* m_A

In [ ]:
A=[]
for i in alldata2.columns:
    if 'A' in i:
        A.append(i)
m_A=alldata2[A].mean(axis=1)

# alldata2['m_A']=m_A
# alldata2

# alldata2['m_E*A']=m_E*m_A

#### TIPI
- Extraversion: 1, 6R; 
- Agreeableness: 2R, 7; 
- Conscientiousness; 3, 8R; 
- Emotional Stability: 4R, 9;
- Openness to Experiences: 5, 10R.

In [ ]:
Extraversion = alldata2['tp01'] + (8 -alldata2['tp06'])
Agreeableness = alldata2['tp07'] + (8 -alldata2['tp02'])
Conscientiousness = alldata2['tp03'] + (8 -alldata2['tp08'])
Emotional_Stability = alldata2['tp09'] + (8 -alldata2['tp04'])
Openness_to_Experiences = alldata2['tp05'] + (8 -alldata2['tp10'])

alldata2['tp_eo'] = Extraversion * Openness_to_Experiences
alldata2['tp_ce'] = Conscientiousness * Emotional_Stability

# for i in range(9):
#     alldata.drop(['tp0{}'.format(i+1)], axis=1, inplace=True)

4. age 데이터 활용

* 나이대별 응답시간, 선호도 평균
- 'age_group','education','married','religion'

In [ ]:
age_A=alldata2.groupby(['married','education'])[A].mean().reset_index()
age_A.columns=['married','education', 'QaA2', 'QbA2', 'QcA2', 'QdA2', 'QeA2','QfA2', 'QgA2', 'QhA2', 'QiA2', 'QjA2', 'QkA2', 'QlA2', 'QmA2', 'QnA2', 'QoA2','QpA2', 'QqA2', 'QrA2', 'QsA2', 'QtA2']
alldata2=alldata2.merge(age_A,on=['married','education'],how='left')

In [ ]:
age_E=alldata2.groupby(['religion','education'])[E].mean().reset_index()
age_E.columns=['religion','education', 'QaE2', 'QbE2', 'QcE2', 'QdE2', 'QeE2','QfE2', 'QgE2', 'QhE2', 'QiE2', 'QjE2', 'QkE2', 'QlE2', 'QmE2', 'QnE2', 'QoE2','QpE2', 'QqE2', 'QrE2', 'QsE2', 'QtE2']
alldata2=alldata2.merge(age_E,on=['education','religion'],how='left')

In [ ]:
age_AE=((alldata2.groupby(['age_group','education','married'])[A].mean().sum(axis=1)/len(A))/(alldata2.groupby(['age_group','education','married'])[E].mean().sum(axis=1)/len(E))).reset_index()
age_AE.columns=['age_group','education','married','age_AE']

alldata2=alldata2.merge(age_AE,on=['age_group','education','married'],how='left')

In [ ]:
p_wf=alldata2.groupby(['education','married','religion','age_group'])['wf_01','wf_02','wf_03'].mean().reset_index()
p_wf.columns=['education','married','religion','age_group','p_wf_01','p_wf_02','p_wf_03']
alldata2=alldata2.merge(p_wf,on=['education','married','religion','age_group'],how='left')

alldata2['s_p_wf']=alldata2['p_wf_01'] * alldata2['p_wf_02'] * alldata2['p_wf_03']

p_wr=alldata2.groupby(['education','married','religion','age_group'])['wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05', 'wr_06', 'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12', 'wr_13'].mean().reset_index()
p_wr.columns=['education','married','religion','age_group','p_wr_01', 'p_wr_02', 'p_wr_03', 'p_wr_04', 'p_wr_05', 'p_wr_06', 'p_wr_07', 'p_wr_08', 'p_wr_09', 'p_wr_10', 'p_wr_11', 'p_wr_12', 'p_wr_13']
p_wr['m_p_wr']=p_wr.iloc[:,3:].std(axis=1)
p_wr.drop(['p_wr_01', 'p_wr_02', 'p_wr_03', 'p_wr_04', 'p_wr_05', 'p_wr_06', 'p_wr_07', 'p_wr_08', 'p_wr_09', 'p_wr_10', 'p_wr_11', 'p_wr_12', 'p_wr_13'],inplace=True,axis=1)
alldata2=alldata2.merge(p_wr,on=['education','married','religion','age_group'],how='left')

In [ ]:
age_Ae=alldata2.groupby(['education'])[A].mean().reset_index()
age_Ae.columns=['education', 'QaAe2', 'QbAe2', 'QcAe2', 'QdAe2', 'QeAe2','QfAe2', 'QgAe2', 'QhAe2', 'QiAe2', 'QjAe2', 'QkAe2', 'QlAe2', 'QmAe2', 'QnAe2', 'QoAe2','QpAe2', 'QqAe2', 'QrAe2', 'QsAe2', 'QtAe2']
alldata2=alldata2.merge(age_Ae,on=['education'],how='left')

age_Am=alldata2.groupby(['married'])[A].mean().reset_index()
age_Am.columns=['married', 'QaAm2', 'QbAm2', 'QcAm2', 'QdAm2', 'QeAm2','QfAm2', 'QgAm2', 'QhAm2', 'QiAm2', 'QjAm2', 'QkAm2', 'QlAm2', 'QmAm2', 'QnAm2', 'QoAm2','QpAm2', 'QqAm2', 'QrAm2', 'QsAm2', 'QtAm2']
alldata2=alldata2.merge(age_Am,on=['married'],how='left')

age_Ar=alldata2.groupby(['religion'])[A].mean().reset_index()
age_Ar.columns=['religion', 'QaAr2', 'QbAr2', 'QcAr2', 'QdAr2', 'QeAr2','QfAr2', 'QgAr2', 'QhAr2', 'QiAr2', 'QjAr2', 'QkAr2', 'QlAr2', 'QmAr2', 'QnAr2', 'QoAr2','QpAr2', 'QqAr2', 'QrAr2', 'QsAr2', 'QtAr2']
alldata2=alldata2.merge(age_Ar,on=['religion'],how='left')

In [ ]:
alldata2['age_family'] = alldata2['age_group'] * alldata2['familysize']
alldata2['edu_engnat'] = alldata2['education'] * alldata2['engnat']
alldata2['race_religion'] = alldata2['race'] * alldata2['religion']
alldata2['QnfA'] = alldata2['QnA'] + alldata2['QfA']
alldata2['tp1_6'] = alldata2['tp01'] / alldata2['tp06']
alldata2['wr_10_12'] = alldata2['wr_12'] + alldata2['wr_10']

* correlation

In [ ]:
# corr = alldata2.corr()

# corr_list = []
# for i in corr.columns:
#     corr_list.append(corr[i][np.abs(corr[i])>0.5][1:].index)

### PolynomialFeatures

* 음수 및 0 처리

In [ ]:
for i in alldata2.columns[alldata2.min()<0]:
    alldata2[i] = alldata2[i] + np.abs(alldata2[i].min())

# for i in alldata2.columns[alldata2.min()==0]:
#     alldata2[i] = alldata2[i] + 0.01

## eval_set

In [ ]:
train2=alldata2.iloc[:len(train),:]
test2=alldata2.iloc[len(train):,:]

In [ ]:
train2 = np.log(train2+0.01)
test2 = np.log(test2+0.01)

## modeling

In [ ]:
x_train,x_valid,y_train,y_valid=train_test_split(train2,train['voted'],stratify=train['voted'],test_size=0.2,random_state=42,shuffle=True)
# skf=StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

* 0.7656539197086892
* wr/wf : 0.7660170017548162<br>
* p_wf : 0.7663123674220365<br>
* age_A + p_wf(4개) : 0.7666193640949511
* age_A('married','education') + p_wf('education','married','religion','age_group') : 0.7670789623995943

In [ ]:
skf1=StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
LGBM1=LGBMClassifier(random_state=42,eval_metric='AUC',num_leaves=31, subsample=0.7, colsample_bytree=0.7
                    , max_depth=-1, learning_rate=0.01, n_estimators = 1000, device='gpu', stopping_rounds = 30)
print(np.mean(cross_val_score(LGBM1,train2,(train['voted']-1),scoring='roc_auc', cv=skf1)))

In [ ]:
LGBM2=LGBMClassifier(random_state=42,eval_metric='AUC',num_leaves=7, subsample=0.7, colsample_bytree=0.7
                    , max_depth=-1, learning_rate=0.01, n_estimators = 1000, device='gpu', stopping_rounds = 30)
print(np.mean(cross_val_score(LGBM2,train2,(train['voted']-1),scoring='roc_auc', cv=skf)))

In [ ]:
LGBM3=LGBMClassifier(random_state=42,eval_metric='AUC',num_leaves=127, subsample=0.7, colsample_bytree=0.7
                    , max_depth=-1, learning_rate=0.01, n_estimators = 1000, device='gpu', stopping_rounds = 30)
print(np.mean(cross_val_score(LGBM3,train2,(train['voted']-1),scoring='roc_auc', cv=skf)))

In [ ]:
skf=StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
y=train['voted']

LGBM=LGBMClassifier(random_state=42,eval_metric='AUC',num_leaves=31, subsample=0.7, colsample_bytree=0.7
                    , max_depth=-1, learning_rate=0.01, n_estimators = 1000, device='gpu', stopping_rounds = 30)
lgbm_result_=0
for train_index, valid_index in skf.split(train2,y) :
#     print(valid_index)
    x_train2=train2.iloc[train_index]
    x_valid2=train2.iloc[valid_index]
    y_train2=y.iloc[train_index]
    y_valid2=y.iloc[valid_index]    
    LGBM.fit(x_train2,y_train2, eval_set=[(x_valid2,y_valid2)],verbose=0)
    lgbm_result_+=LGBM.predict_proba(test2)/10

# pd.DataFrame(cat_result).to_csv('cat_predict.csv', index=False)
pd.DataFrame(lgbm_result_).to_csv('lgbm_predict_.csv', index=False)

In [ ]:
skf=StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
y=train['voted']

LGBM_v=LGBMClassifier(random_state=42,eval_metric='AUC',num_leaves=31, subsample=0.7, colsample_bytree=0.7
                    , max_depth=-1, learning_rate=0.01, n_estimators = 1000, device='gpu', stopping_rounds = 30)
lgbm_result_v=0
for train_index, valid_index in skf.split(x_train,y_train) :
#     print(valid_index)
    x_train2=train2.iloc[train_index]
    x_valid2=train2.iloc[valid_index]
    y_train2=y.iloc[train_index]
    y_valid2=y.iloc[valid_index]    
    LGBM_v.fit(x_train2,y_train2, eval_set=[(x_valid2,y_valid2)],verbose=0)
    lgbm_result_v+=LGBM_v.predict_proba(x_valid)/10

In [ ]:
skf=StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
y=train['voted']

LGBM_7=LGBMClassifier(random_state=42,eval_metric='AUC',num_leaves=7, subsample=0.7, colsample_bytree=0.7
                    , max_depth=8, learning_rate=0.01, n_estimators = 1000, device='gpu', stopping_rounds = 30)
lgbm_result_7=0
for train_index, valid_index in skf.split(train2,train['voted']) :
#     print(valid_index)
    x_train2=train2.iloc[train_index]
    x_valid2=train2.iloc[valid_index]
    y_train2=y.iloc[train_index]
    y_valid2=y.iloc[valid_index]    
    LGBM_7.fit(x_train2,y_train2, eval_set=[(x_valid2,y_valid2)],verbose=0)
    lgbm_result_7+=LGBM_7.predict_proba(test2)/10
    
LGBM_127=LGBMClassifier(random_state=42,eval_metric='AUC',num_leaves=127, subsample=0.7, colsample_bytree=0.7
                    , max_depth=8, learning_rate=0.01, n_estimators = 1000, device='gpu', stopping_rounds = 30)
lgbm_result_127=0
for train_index, valid_index in skf.split(train2,train['voted']) :
#     print(valid_index)
    x_train2=train2.iloc[train_index]
    x_valid2=train2.iloc[valid_index]
    y_train2=y.iloc[train_index]
    y_valid2=y.iloc[valid_index]    
    LGBM_127.fit(x_train2,y_train2, eval_set=[(x_valid2,y_valid2)],verbose=0)
    lgbm_result_127+=LGBM_127.predict_proba(test2)/10

In [ ]:
roc_auc_score(y_valid, cat_result2[:,1])

In [ ]:
roc_auc_score(y_valid, lgbm_result2[:,1])

## submission

In [ ]:
predict=(lgbm_result_[:,1] * 0.4) + (lgbm_result_7[:,1] * 0.3) + (lgbm_result_127[:,1] * 0.3)
sub['voted']=predict
sub.to_csv('suba.csv',index=False)

In [ ]:
sub['voted']=lgbm_result_[:,1]
sub.to_csv('sub.csv',index=False)
sub